In [ ]:
import streamlit as st
import random
import requests
from bs4 import BeautifulSoup
from typing import Dict, Set, Tuple

# Web Scraping Function
def get_powerball_results():
    url = "https://www.thelott.com/powerball/results"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Scrape jackpot amount
    jackpot_div = soup.find("div", {"class": "jackpot-amount"})
    jackpot_amount = float(jackpot_div.text.strip().replace('$', '').replace(',', ''))

    # Scrape prize pool amounts
    prize_divs = soup.find_all("div", {"class": "prize-pool-amount"})
    prize_amounts = [float(div.text.strip().replace('$', '').replace(',', '')) for div in prize_divs]

    # Define prize pool categories (update according to the actual structure)
    prize_pool_categories = ["7+P", "7", "6+P", "6", "5+P", "5", "4+P", "3+P", "2+P"]

    prize_pool_amounts = dict(zip(prize_pool_categories, prize_amounts))

    return jackpot_amount, prize_pool_amounts

# Get the latest PowerBall results
JACKPOT_AMOUNT, PRIZE_POOL_AMOUNTS = get_powerball_results()
TICKET_COST = 1.35  # Placeholder for the cost of each ticket (you can scrape this too if needed)

# Function to calculate the win amount
def calc_win_amt(my_numbers: Dict[str, Set[int]], winning_numbers: Dict[str, Set[int]], times_won: Dict[str, int]) -> float:
    win_amt = 0
    white_matches = len(my_numbers['whites'].intersection(winning_numbers['whites']))
    power_matches = my_numbers['red'] == winning_numbers['red']

    if white_matches == 7:
        if power_matches:
            win_amt = PRIZE_POOL_AMOUNTS["7+P"] + JACKPOT_AMOUNT
            times_won['7+P'] += 1
        else:
            win_amt = PRIZE_POOL_AMOUNTS["7"]
            times_won['7'] += 1
    elif white_matches == 6:
        if power_matches:
            win_amt = PRIZE_POOL_AMOUNTS["6+P"]
            times_won['6+P'] += 1
        else:
            win_amt = PRIZE_POOL_AMOUNTS["6"]
            times_won['6'] += 1
    elif white_matches == 5:
        if power_matches:
            win_amt = PRIZE_POOL_AMOUNTS["5+P"]
            times_won['5+P'] += 1
        else:
            win_amt = PRIZE_POOL_AMOUNTS["5"]
            times_won['5'] += 1
    elif white_matches == 4 and power_matches:
        win_amt = PRIZE_POOL_AMOUNTS["4+P"]
        times_won['4+P'] += 1
    elif white_matches == 3 and power_matches:
        win_amt = PRIZE_POOL_AMOUNTS["3+P"]
        times_won['3+P'] += 1
    elif white_matches == 2 and power_matches:
        win_amt = PRIZE_POOL_AMOUNTS["2+P"]
        times_won['2+P'] += 1

    return win_amt

# Function to simulate the PowerBall lottery
def simulate_powerball(tickets_per_game: int, num_games: int) -> Tuple[float, float, Dict[str, int]]:
    total_spent = 0
    earnings = 0
    times_won = {  # Initialize the win counters for each category
        "7+P": 0,
        "7": 0,
        "6+P": 0,
        "6": 0,
        "5+P": 0,
        "5": 0,
        "4+P": 0,
        "3+P": 0,
        "2+P": 0
    }

    # Loop through the number of games
    for _ in range(num_games):
        # Generate the winning numbers for the current game
        white_drawing = set(random.sample(list(range(1, 36)), 7))
        red_drawing = random.choice(list(range(1, 21)))

        winning_numbers = {'whites': white_drawing, 'red': red_drawing}

        # Loop through the number of tickets per game
        for _ in range(tickets_per_game):
            total_spent += TICKET_COST
            my_whites = set(random.sample(list(range(1, 36)), 7))
            my_red = random.choice(list(range(1, 21)))

            my_numbers = {'whites': my_whites, 'red': my_red}

            win_amt = calc_win_amt(my_numbers, winning_numbers, times_won)
            earnings += win_amt

    return total_spent, earnings, times_won

# Streamlit app
st.title("PowerBall Simulator")
st.write("Welcome to the PowerBall Simulator!")

# Get user input for the number of tickets per game and number of games
tickets_per_game = st.number_input("How many tickets would you like to purchase?", min_value=1, value=1)
num_games = st.number_input("How many games would you like to play?", min_value=1, value=1)

if st.button("Run Simulation"):
    total_spent, earnings, times_won = simulate_powerball(tickets_per_game, num_games)
    st.write(f'Spent: ${total_spent:.2f}')
    st.write(f'Earnings: ${earnings:.2f}')

    # Convert the times_won dictionary to a table format
    results_table = [
        {"Div": "1", "Num Tickets Won": times_won["7+P"]},
        {"Div": "2", "Num Tickets Won": times_won["7"]},
        {"Div": "3", "Num Tickets Won": times_won["6+P"]},
        {"Div": "4", "Num Tickets Won": times_won["6"]},
        {"Div": "5", "Num Tickets Won": times_won["5+P"]},
        {"Div": "6", "Num Tickets Won": times_won["5"]},
        {"Div": "7", "Num Tickets Won": times_won["4+P"]},
        {"Div": "8", "Num Tickets Won": times_won["3+P"]},
        {"Div": "9", "Num Tickets Won": times_won["2+P"]}
    ]

    st.table(results_table)


ModuleNotFoundError: No module named 'streamlit'